article class = sc-1ww42at-0, EyLft qjwil1-0 iCnXma

In [35]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import json
import folium
import branca.colormap as cm

In [36]:
# send requests

url = 'https://www.immoscout24.ch/de/wohnung/mieten/ort-luzern?pn=1&r=6'

page = requests.get(url)

soup = BeautifulSoup(page.content, 'html.parser')

In [37]:
# read out table

raw_data = soup.find(id='state')

jason_data = json.loads(raw_data.string[25:])
property_list = jason_data['pages']['searchResult']['resultData']['allProperties']
parsed_list = []

for elem in property_list:
    #print('----------')
    #print('{} {}'.format(elem['id'], elem['priceFormatted']['value'][4:-4]))
    price =  elem['priceFormatted']['value']
    parsed_list.append([elem['id'], price, elem['latitude'], elem['longitude']])
    #property_dict.append(elem)

#parsed_list

In [38]:
property_df = pd.DataFrame(parsed_list)
property_df.columns = ['Id', 'Price', 'Latitude', 'Longitude']

filter = property_df['Price'] != 'Preis auf Anfrage'
property_df = property_df[filter]

property_df['Price'] = property_df['Price'].str[4:-2]
property_df['Price'] = property_df['Price'].str.replace('\'', '')
property_df['Price'] = property_df['Price'].astype('int64')

property_df.describe()

,Id,Price,Latitude,Longitude
count,6.990000e+02,699.000000,699.000000,699.000000
mean,5.762488e+06,1910.690987,47.053289,8.302386
std,4.584244e+05,783.756023,0.019668,0.026927
min,6.020960e+05,130.000000,46.994750,8.230550
25%,5.795507e+06,1490.000000,47.038370,8.282420
50%,5.914668e+06,1740.000000,47.052210,8.301000
75%,5.948961e+06,2165.000000,47.068040,8.313595
max,5.972088e+06,7900.000000,47.097020,8.380820


In [39]:
# create map of lucerne using latitude and longitude values
map_lucerne = folium.Map(location=[47.05, 8.3], zoom_start=12.5)

linear = cm.LinearColormap(
    ['green', 'red'],
    vmin=100, vmax=8000
)

linear

# add markers to map
for lat, lng, price in zip(property_df['Latitude'], property_df['Longitude'], property_df['Price']):
    label = folium.Popup(str(price), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=linear(price),
        fill=True,
        fill_color=linear(price),
        fill_opacity=0.7,
        parse_html=False).add_to(map_lucerne)

map_lucerne  